In [1]:
import json
from typing import Dict
import pandas as pd
from pandas import DataFrame
from tabulate import tabulate
from Classes import Player
from PlayerParser import parse_players
from WeightCalculator import WeightCalculator

players: Dict[str, Player] = parse_players([
    "Warlords.Games_Information_CTF_11-7-2024.json",
    "Warlords.Games_Information_CTF_11-3-2023.json",
    "Warlords.Games_Information_CTF_4-8-2023.json",
    "Warlords.Games_Information_CTF_4-4-2024.json",
    "Warlords.Games_Information_CTF_7-1-2024.json",
    "Warlords.Games_Information_CTF_10-4-2022.json",
])

all_stats = []
for player_name, player in players.items():
    player_stats = player.get_player_stats()
    player_stats["name"] = player.name
    all_stats.append(player_stats)

Unique games:  2675
Valid games:  1217


In [2]:
import json

serialized_data = json.dumps(all_stats)
with open("Data", 'w') as file:
    file.write(serialized_data)

print(f"Data Serialized")


Data Serialized


In [1]:
import json

with open("Data", 'r') as file:
    serialized_data = file.read()

all_stats = json.loads(serialized_data)
print(f"Data Deserialized")

Data Deserialized


In [2]:
import pandas as pd
from pandas import DataFrame

df: DataFrame = pd.DataFrame(all_stats)

In [3]:
from WeightCalculator import WeightCalculator


def drop_correlated(df: DataFrame) -> DataFrame:
    df = df.copy()
    df.drop(
        columns=[
            "kills_average", "kills_average", "assists_average", "deaths_average", "damage_average", "healing_average", "absorbed_average", "damage_on_carrier_average",
            "healing_on_carrier_average", "flag_returns_average", "blocks_travelled_average"
        ],
        inplace=True
    )
    return df


def average_respawn_time(df: DataFrame) -> DataFrame:
    df = df.copy()
    df["average_respawn_time"] = df["total_deaths_average"] / df["seconds_in_respawn_average"]
    df.drop(
        columns=["total_deaths_average", "seconds_in_respawn_average"],
        inplace=True
    )
    return df


def average_dhp(df: DataFrame) -> DataFrame:
    df = df.copy()
    df["average_dhp"] = (df["total_damage_average"] + df["total_absorbed_average"] + df["total_healing_average"]) / 3
    df.drop(
        columns=["total_damage_average", "total_absorbed_average", "total_healing_average"], inplace=True
    )
    return df


def average_respawn_time_kda(df: DataFrame) -> DataFrame:
    df = df.copy()
    df["average_respawn_time"] = df["total_deaths_average"] / df["seconds_in_respawn_average"]
    df["kda"] = (df["total_kills_average"] + df["total_assists_average"]) / df["total_deaths_average"]
    df.drop(
        columns=["seconds_in_respawn_average", "total_kills_average", "total_assists_average", "total_deaths_average"], inplace=True
    )
    return df


def kda(df: DataFrame) -> DataFrame:
    df = df.copy()
    df["kda"] = (df["total_kills_average"] + df["total_assists_average"]) / df["total_deaths_average"]
    df.drop(
        columns=["total_kills_average", "total_assists_average", "total_deaths_average"], inplace=True
    )
    return df


def ka(df: DataFrame) -> DataFrame:
    df = df.copy()
    df["ka"] = df["total_kills_average"] + df["total_assists_average"]
    df.drop(
        columns=["total_kills_average", "total_assists_average"], inplace=True
    )
    return df

print("---")
WeightCalculator([drop_correlated, average_respawn_time], polynomial_degree=2).train_best(df)
print("---")
WeightCalculator([drop_correlated, average_respawn_time, average_dhp], polynomial_degree=2).train_best(df)
print("---")
WeightCalculator([drop_correlated, average_respawn_time_kda, average_dhp], polynomial_degree=2).train_best(df)
print("---")
WeightCalculator([drop_correlated, average_respawn_time, average_dhp, ka], polynomial_degree=2).train_best(df)



---

drop_correlated + average_respawn_time:
- 5-Fold Cross-Validation Mean Squared Error (MSE): 0.5603 (±0.6473)
- 5-Fold Cross-Validation R-squared (R2): -79.8613 (±108.1811)

Test Set Performance:
- Mean Squared Error (MSE): 0.2539
- R-squared (R2): -26.6440

- Coefficients (sorted by absolute value):
  - seconds_in_combat_average total_healing_average: 0.2834
  - total_damage_average total_healing_average: -0.2274
  - total_damage_average total_absorbed_average: -0.2027
  - total_assists_average total_absorbed_average: 0.1896
  - seconds_in_combat_average total_damage_average: 0.1386
  - total_damage_on_carrier_average total_assists_average: 0.1243
  - total_healing_average: 0.1170
  - seconds_in_combat_average total_kills_average: -0.1153
  - total_damage_on_carrier_average average_respawn_time: 0.1146
  - seconds_in_combat_average^2: -0.1089
  - total_kills_average total_healing_average: 0.1026
  - flag_captures_average total_assists_average: -0.1025
  - total_healing_average^2: 

In [ ]:
# from typing import Dict
# import pandas as pd
# from Classes import Player, PlayerStats
# from PlayerParser import parse_game_players
# 
# players: Dict[PlayerStats, int] = parse_game_players([
#     "Warlords.Games_Information_CTF_11-7-2024.json",
#     "Warlords.Games_Information_CTF_11-3-2023.json",
#     "Warlords.Games_Information_CTF_4-8-2023.json",
#     "Warlords.Games_Information_CTF_4-4-2024.json",
#     "Warlords.Games_Information_CTF_7-1-2024.json",
# ])
# print(players)
